# Descrizione notebook

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
import math
import random
from itertools import combinations
from ipynb.fs.defs.creazione_CSV import normalization
from ipynb.fs.defs.UBCF import pearson_correlation_coefficient, score_user_item_ubcf, recommendation_ubcf
from ipynb.fs.defs.IBCF import cosine_similarity, score_user_item_ibcf, recommendation_ibcf

In [2]:
pd.set_option('display.max_columns', 102)
# lettura della matrice con gli utenti che hanno valutato tutte le barzellette
complete_ratings = pd.read_csv('./data/complete_ratings_matrix.csv')

complete_ratings = complete_ratings.iloc[:50, :]

# lettura dei rating non normalizzati dal relativo CSV (serve solo per calcolare la predizione, nella
# quale serve calcolare il rating medio dell'utente)
no_normalized_ratings_df = pd.read_csv('./data/no_normalized_ratings.csv')
complete_ratings

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,7.82,-3.83,8.64,8.98,-4.32,-3.20,-4.90,-0.92,-1.46,3.64,3.16,9.03,0.97,-1.31,-6.50,-3.20,8.64,-2.14,0.10,9.03,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,-8.83,-7.91,1.07,-1.89,-6.36,-4.22,-9.27,-9.81,-7.82,-0.63,-6.31,-7.48,-6.07,-3.45,-3.20,0.53,-9.27,-9.56,-5.49,-1.12,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
2,8,100,6.84,3.16,9.17,-6.21,-8.16,-1.70,9.27,1.41,-5.19,-4.42,8.20,-7.86,-6.94,-7.96,0.29,-9.90,-7.09,-7.18,1.02,-0.29,-4.71,-7.43,1.12,-8.25,3.79,0.10,3.45,-3.45,7.57,-4.17,-0.44,9.27,5.83,-3.40,-5.44,1.60,-7.14,3.01,-6.07,5.68,-6.46,-4.42,-8.98,0.53,-1.26,-4.42,-8.93,8.93,-0.10,7.43,-4.51,-7.38,4.17,-9.03,-2.14,1.31,-5.68,-9.08,-6.21,2.48,-3.20,-6.12,-2.91,-9.22,-2.62,-7.33,-3.25,-4.22,-9.81,-2.09,-9.90,-3.79,0.49,-7.86,-9.85,1.80,-4.17,-2.38,-9.13,-8.88,3.54,-4.56,-5.44,-5.97,-9.90,-0.34,-4.13,-0.10,-0.24,-7.96,7.23,-1.12,-0.10,-5.68,-3.16,-3.35,2.14,-0.05,1.31,0.00
3,9,100,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93,-7.86,-1.60,-2.91,-0.29,-4.85,-0.49,-8.74,-6.99,-8.74,-2.91,-3.35,-0.29,3.98,-1.60,-0.29,1.21,3.79,2.91,4.85,2.28,0.97,-0.29,6.36,0.53,-0.29,4.66,3.35,0.97,-3.11,-1.84,4.42,-4.42,-1.41,1.41,-5.34,-6.99,1.41,5.10,1.84,-0.29,0.53,-0.29,-6.99,-3.54,6.36,1.84,3.35,-0.29,-0.29,-5.29,-6.89,-5.29,1.21,5.53,1.41,-6.50,4.85,4.85,2.62,1.41,6.36,1.41,-4.76,4.17,-2.04,-6.31,-0.29,-0.29,0.78,-0.29,-0.29,-0.29,-0.29,-3.20,-0.29,-3.20,-3.40,-0.29,-0.29,5.73,4.56,-2.23,4.37,-0.29,4.17,-0.29,-0.29,-0.29,-0.29,-0.29,-3.40,-4.95
4,12,100,1.31,1.80,2.57,-2.38,0.73,0.73,-0.97,5.00,-7.23,-1.36,3.83,1.75,5.63,-2.86,-1.80,-2.04,5.53,-0.29,-0.58,1.36,2.43,0.63,-4.61,-4.37,-1.65,-1.36,7.57,-0.15,5.97,-3.50,0.24,4.27,-0.68,-3.79,7.62,5.15,2.43,0.10,0.83,-2.04,0.53,2.33,-2.28,-5.15,-0.58,-1.26,2.04,-1.41,2.38,6.60,2.04,-1.99,-2.77,-0.49,1.75,0.92,-6.89,-5.29,0.68,-2.38,-0.15,-0.24,1.46,-2.14,2.43,-2.43,-5.10,-0.49,0.15,-2.62,-2.91,1.12,1.70,0.00,2.04,3.06,-4.47,2.04,1.12,3.30,1.46,0.68,-2.57,-0.63,0.58,-0.63,1.89,2.38,6.21,-2.52,1.46,1.70,0.29,-3.30,3.45,5.44,4.08,2.48,4.51,4.66
5,14,100,9.22,9.27,9.22,8.30,7.43,0.44,3.50,8.16,5.97,8.98,3.74,5.87,8.69,6.31,1.07,-9.13,3.69,-7.33,3.88,-7.48,8.45,9.17,9.22,9.03,9.27,4.42,8.69,1.75,7.57,-7.14,8.93,6.80,0.87,7.23,8.79,8.79,7.43,8.59,8.54,9.17,8.20,8.54,8.20,9.17,9.08,2.86,-2.38,7.43,1.17,-0.29,5.00,8.35,7.62,8.16,8.06,8.35,9.22,-9.27,7.14,9.08,4.08,-9.81,-7.14,

# Preparazione Training set e Test set

In [3]:
test_set = []
no_normalized_train_set = complete_ratings.copy(deep=True)  
# per ogni riga (<indice_riga, riga>) e per ogni barzelletta della riga si salva la
# coppia <nome_barzelletta, rating> in una lista (altrimenti si perde il riferimento al nome della barzelletta)
# poi si prendono n coppie random, si salvano le informazioni nel test set, e si toglie dalla matrice completa
# quello che si è preso almeno si ottiene il training set di conseguenza
for index, row in no_normalized_train_set.iterrows():
    lst_app = []
    for i, elem in row[2:].items():
        lst_app.append((i, elem))
    lst_pair = random.sample(lst_app, 20)
    # pair = random.choice(lst_app)
    for pair in lst_pair:
        app = {}
        app["user_id"] = int(row["user_id"])
        app[pair[0]] = pair[1]
        test_set.append(app)
        no_normalized_train_set.loc[no_normalized_train_set["user_id"] == int(row["user_id"]), pair[0]] = np.nan
    
# test_set_df = pd.DataFrame(test_set)

In [4]:
no_normalized_train_set

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,NaN,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,NaN,9.22,6.75,8.64,NaN,7.43,4.56,NaN,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,NaN,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,7.82,-3.83,8.64,8.98,-4.32,-3.20,-4.90,NaN,-1.46,NaN,3.16,9.03,0.97,-1.31,NaN,-3.20,NaN,-2.14,0.10,9.03,-6.70,-3.35,-9.03,4.47,4.08,NaN,NaN,1.12,0.78,NaN,NaN,2.77,8.30,NaN,7.33,NaN,7.72,NaN,8.64,8.20,3.93,NaN,NaN,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,NaN,NaN,1.07
1,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,NaN,-6.80,NaN,-5.00,-8.59,0.49,-8.93,-3.69,NaN,-2.28,NaN,-3.01,-0.58,-2.38,NaN,-2.23,NaN,-5.24,-5.53,-0.68,NaN,-3.93,NaN,NaN,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,-8.83,-7.91,NaN,-1.89,-6.36,-4.22,NaN,-9.81,NaN,-0.63,NaN,-7.48,-6.07,-3.45,-3.20,NaN,NaN,-9.56,-5.49,-1.12,-6.65,-2.86,-4.61,NaN,-7.91,-9.81,NaN,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,NaN,-5.68,-4.71,-3.54,-6.89,NaN,-2.96,-2.18,-3.35,NaN,-9.08,-5.05,-3.45
2,8,100,NaN,3.16,NaN,-6.21,-8.16,NaN,9.27,1.41,-5.19,-4.42,8.20,-7.86,-6.94,NaN,0.29,-9.90,-7.09,-7.18,1.02,-0.29,-4.71,-7.43,1.12,-8.25,3.79,0.10,NaN,-3.45,7.57,-4.17,-0.44,9.27,5.83,-3.40,-5.44,1.60,-7.14,3.01,-6.07,5.68,-6.46,-4.42,-8.98,NaN,-1.26,-4.42,-8.93,8.93,-0.10,7.43,-4.51,-7.38,4.17,NaN,-2.14,1.31,-5.68,-9.08,NaN,2.48,-3.20,-6.12,-2.91,-9.22,-2.62,-7.33,-3.25,-4.22,-9.81,NaN,-9.90,NaN,0.49,-7.86,-9.85,1.80,NaN,NaN,-9.13,-8.88,NaN,-4.56,NaN,-5.97,-9.90,-0.34,NaN,NaN,NaN,-7.96,7.23,-1.12,-0.10,NaN,-3.16,NaN,2.14,NaN,1.31,0.00
3,9,100,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93,-7.86,NaN,-2.91,-0.29,-4.85,-0.49,-8.74,-6.99,-8.74,-2.91,NaN,-0.29,3.98,-1.60,-0.29,1.21,NaN,2.91,NaN,NaN,0.97,-0.29,6.36,0.53,NaN,4.66,3.35,0.97,NaN,-1.84,4.42,-4.42,-1.41,1.41,-5.34,-6.99,1.41,5.10,1.84,-0.29,0.53,-0.29,-6.99,NaN,6.36,1.84,3.35,NaN,NaN,-5.29,-6.89,-5.29,1.21,5.53,NaN,-6.50,4.85,4.85,2.62,1.41,6.36,1.41,NaN,4.17,NaN,NaN,-0.29,NaN,0.78,-0.29,-0.29,-0.29,-0.29,-3.20,-0.29,-3.20,-3.40,-0.29,NaN,5.73,NaN,-2.23,NaN,-0.29,4.17,NaN,-0.29,-0.29,-0.29,-0.29,-3.40,NaN
4,12,100,1.31,1.80,2.57,-2.38,0.73,0.73,-0.97,5.00,-7.23,-1.36,3.83,1.75,NaN,-2.86,-1.80,-2.04,5.53,NaN,NaN,1.36,2.43,0.63,-4.61,-4.37,-1.65,-1.36,NaN,-0.15,5.97,NaN,0.24,4.27,-0.68,-3.79,7.62,5.15,2.43,NaN,0.83,-2.04,0.53,2.33,-2.28,NaN,-0.58,-1.26,NaN,NaN,2.38,NaN,2.04,-1.99,-2.77,-0.49,1.75,0.92,-6.89,-5.29,0.68,NaN,-0.15,NaN,NaN,NaN,2.43,-2.43,NaN,-0.49,0.15,-2.62,-2.91,1.12,1.70,0.00,2.04,3.06,-4.47,2.04,1.12,3.30,NaN,0.68,-2.57,-0.63,0.58,-0.63,1.89,2.38,NaN,-2.52,1.46,1.70,0.29,NaN,NaN,NaN,4.08,2.48,4.51,4.66
5,14,100,9.22,9.27,NaN,8.30,7.43,0.44,3.50,NaN,5.97,8.98,3.74,5.87,8.69,6.31,1.07,NaN,3.69,-7.33,NaN,-7.48,NaN,9.17,9.22,9.03,9.27,4.42,8.69,1.75,7.57,-7.14,8.93,6.80,0.87,7.23,8.79,8.79,7.43,8.59,8.54,9.17,8.20,8.54,NaN,9.17,9.08,2.86,-2.38,NaN,1.17,-0.29,5.00,8.35,7.62,8.16,NaN,8.35,9.22,-9.27,7.14,9.08,NaN,-9.81,-7.14,9.32,7.14,7.57,-5.78,NaN,NaN,6.36,-7.57,8.93,0.15,8.93,-6.21,NaN,NaN,8.69,-8.45,NaN,8.88,-8.69,7.77,-0.83,NaN,NaN,5.44,NaN,8.45,2.09,8.11,-1.02,5.58,6.84,NaN,-5.92,NaN,8.98

In [5]:
normalizated_train_set = no_normalized_train_set.copy()   
normalizated_train_set = normalization(normalizated_train_set)
normalizated_train_set

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,NaN,-2.994750,3.655250,1.665250,-5.084750,-12.364750,-3.434750,-8.044750,NaN,6.515250,4.045250,5.935250,NaN,4.725250,1.855250,NaN,1.955250,-3.384750,0.595250,-3.914750,-1.834750,5.935250,5.645250,6.465250,-2.654750,4.865250,2.005250,NaN,-3.094750,4.285250,3.795250,-3.624750,4.435250,6.325250,-4.504750,-1.974750,4.385250,0.695250,-3.574750,5.205250,5.115250,-6.534750,5.935250,6.275250,-7.024750,-5.904750,-7.604750,NaN,-4.164750,NaN,0.455250,6.325250,-1.734750,-4.014750,NaN,-5.904750,NaN,-4.844750,-2.604750,6.325250,-9.404750,-6.054750,-11.734750,1.765250,1.375250,NaN,NaN,-1.584750,-1.924750,NaN,NaN,0.065250,5.595250,NaN,4.625250,NaN,5.015250,NaN,5.935250,5.495250,1.225250,NaN,NaN,3.365250,6.275250,1.805250,-2.754750,0.985250,1.855250,-2.124750,0.115250,-7.654750,-2.994750,5.155250,-2.894750,-4.844750,0.355250,NaN,NaN,-1.634750
1,6,100,-1.200875,1.429125,5.409125,-3.530875,-2.120875,0.649125,-3.720875,4.099125,-1.680875,NaN,-1.830875,NaN,-0.030875,-3.620875,5.459125,-3.960875,1.279125,NaN,2.689125,NaN,1.959125,4.389125,2.589125,NaN,2.739125,NaN,-0.270875,-0.560875,4.289125,NaN,1.039125,NaN,NaN,-0.520875,3.419125,4.479125,-1.240875,-3.090875,-0.420875,-2.070875,-3.860875,-2.940875,NaN,3.079125,-1.390875,0.749125,NaN,-4.840875,NaN,4.339125,NaN,-2.510875,-1.100875,1.519125,1.769125,NaN,NaN,-4.590875,-0.520875,3.849125,-1.680875,2.109125,0.359125,NaN,-2.940875,-4.840875,NaN,-4.010875,3.029125,-2.020875,-4.690875,-0.220875,-0.030875,0.549125,-2.310875,-1.530875,0.649125,-1.970875,-4.880875,5.699125,-1.240875,2.109125,-2.650875,-1.150875,4.679125,3.559125,1.039125,NaN,-0.710875,0.259125,1.429125,-1.920875,NaN,2.009125,2.789125,1.619125,NaN,-4.110875,-0.080875,1.519125
2,8,100,NaN,5.733750,NaN,-3.636250,-5.586250,NaN,11.843750,3.983750,-2.616250,-1.846250,10.773750,-5.286250,-4.366250,NaN,2.863750,-7.326250,-4.516250,-4.606250,3.593750,2.283750,-2.136250,-4.856250,3.693750,-5.676250,6.363750,2.673750,NaN,-0.876250,10.143750,-1.596250,2.133750,11.843750,8.403750,-0.826250,-2.866250,4.173750,-4.566250,5.583750,-3.496250,8.253750,-3.886250,-1.846250,-6.406250,NaN,1.313750,-1.846250,-6.356250,11.503750,2.473750,10.003750,-1.936250,-4.806250,6.743750,NaN,0.433750,3.883750,-3.106250,-6.506250,NaN,5.053750,-0.626250,-3.546250,-0.336250,-6.646250,-0.046250,-4.756250,-0.676250,-1.646250,-7.236250,NaN,-7.326250,NaN,3.063750,-5.286250,-7.276250,4.373750,NaN,NaN,-6.556250,-6.306250,NaN,-1.986250,NaN,-3.396250,-7.326250,2.233750,NaN,NaN,NaN,-5.386250,9.803750,1.453750,2.473750,NaN,-0.586250,NaN,4.713750,NaN,3.883750,2.573750
3,9,100,-2.923750,-2.673750,-8.553750,-6.023750,-7.873750,0.576250,-4.423750,-8.063750,-6.993750,NaN,-2.043750,0.576250,-3.983750,0.376250,-7.873750,-6.123750,-7.873750,-2.043750,NaN,0.576250,4.846250,-0.733750,0.576250,2.076250,NaN,3.776250,NaN,NaN,1.836250,0.576250,7.226250,1.396250,NaN,5.526250,4.216250,1.836250,NaN,-0.973750,5.286250,-3.553750,-0.543750,2.276250,-4.473750,-6.123750,2.276250,5.966250,2.706250,0.576250,1.396250,0.576250,-6.123750,NaN,7.226250,2.706250,4.216250,NaN,NaN,-4.423750,-6.023750,-4.423750,2.076250,6.396250,NaN,-5.633750,5.716250,5.716250,3.486250,2.276250,7.226250,2.276250,NaN,5.036250

# Valutazione MAE

In [6]:
def mae(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        if joke in i and joke in j:
            t1+=abs(i[joke] - j[joke])
    return t1 / len(final)

# Valutazione RMSE

In [7]:
def rmse(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        if joke in i and joke in j:
            t1+=(i[joke] - j[joke]) * (i[joke] - j[joke])
    return np.sqrt(t1 / len(final))

# Raccomandazione UBCF

In [8]:
final = []

# per ogni utente nel training set
for n in list(normalizated_train_set["user_id"]):
    lst = recommendation_ubcf(n, normalizated_train_set, no_normalized_train_set)
    final = final + lst

L'utente target è: 
   user_id  number_of_jokes_rated  joke_1   joke_2   joke_3   joke_4   joke_5  \
0        2                    100     NaN -2.99475  3.65525  1.66525 -5.08475   

     joke_6   joke_7   joke_8  joke_9  joke_10  joke_11  joke_12  joke_13  \
0 -12.36475 -3.43475 -8.04475     NaN  6.51525  4.04525  5.93525      NaN   

   joke_14  joke_15  joke_16  joke_17  joke_18  joke_19  joke_20  joke_21  \
0  4.72525  1.85525      NaN  1.95525 -3.38475  0.59525 -3.91475 -1.83475   

   joke_22  joke_23  joke_24  joke_25  joke_26  joke_27  joke_28  joke_29  \
0  5.93525  5.64525  6.46525 -2.65475  4.86525  2.00525      NaN -3.09475   

   joke_30  joke_31  joke_32  joke_33  joke_34  joke_35  joke_36  joke_37  \
0  4.28525  3.79525 -3.62475  4.43525  6.32525 -4.50475 -1.97475  4.38525   

   joke_38  joke_39  joke_40  joke_41  joke_42  joke_43  joke_44  joke_45  \
0  0.69525 -3.57475  5.20525  5.11525 -6.53475  5.93525  6.27525 -7.02475   

   joke_46  joke_47  joke_48  joke_49  jok

NameError: name 'jokes_df' is not defined

# Raccomandazione IBCF

In [ ]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(no_normalized_train_set.columns))[2:]
jokes_pairs = list(combinations(all_jokes, 2))
similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalizated_train_set[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosine_similarity(app[i_1], app[i_2])])
    
df = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])
df = df[df['similarity'] > 0.3]
df

In [ ]:
final_2 = []
for n in list(complete_ratings["user_id"]):
    lst = recommendation_ibcf(n, no_normalized_train_set, df)
    final_2 = final_2 + lst

In [ ]:
# final

In [ ]:
test_set.sort(key=lambda x: (x["user_id"],int(list(x.keys())[1][5:])), reverse=False)
#test_set

# Confronto: UBCF vs. ITCF

In [ ]:
print("MAE UBCF: " + str(mae(final, test_set)))
print("RMSE UBCF: " + str(rmse(final, test_set)))

print("MAE IBCF: " + str(mae(final_2, test_set)))
print("RMSE IBCF: " + str(rmse(final_2, test_set)))